<center><h1>Read Data JSON to CSV</h1></center>
<hr>

This notebook is for reading JSON format Million Playlist Dataset(MPD) to CSV format. Also split database for 'Playlist' and 'Track'. 

## References

## Table of Contents <a id="top"></a>

1. [Load Data](#1)
2. [Collect Featrues to Save](#2)
3. [Database Modeling](#3)

<hr>

# 1. Load Data <a id="1"></a>
<p style="text-align:right;"><a href="#top">top 🔝</a></p>

In [1]:
import json
from tqdm import tqdm
import pandas as pd

In [2]:
def load_data(filepath):
    with open(filepath, 'r') as f:
        json_str = ''
        for line in tqdm(f.read()):
            json_str += line
        data = json.loads(json_str)
    return data

In [3]:
filepath = './data/mpd.slice.0-999.json'
data = load_data(filepath)

100%|██████████| 34119368/34119368 [00:11<00:00, 2986796.00it/s]


# 2. Collect Featrues to Save <a id="2"></a>
<p style="text-align:right;"><a href="#top">top 🔝</a></p>

In [4]:
def get_features(data):
    playlist_key = set()
    track_key = set()
    for playlist in data['playlists']:
        for kp in playlist.keys():
            playlist_key.add(kp)

        for track in playlist['tracks']:
            for kt in track.keys():
                track_key.add(kt)

    return playlist_key, track_key

In [5]:
playlist_key, track_key = get_features(data)

In [6]:
print('Key for playlist({})\n{}\n\nKey for track({})\n{}'.format(
    len(playlist_key), playlist_key, len(track_key), track_key)
)

Key for playlist(12)
{'modified_at', 'collaborative', 'duration_ms', 'num_edits', 'num_artists', 'tracks', 'num_tracks', 'pid', 'num_albums', 'name', 'description', 'num_followers'}

Key for track(8)
{'album_name', 'pos', 'artist_name', 'album_uri', 'duration_ms', 'track_name', 'track_uri', 'artist_uri'}


# 3. Database Modeling <a id="3"></a>
<p style="text-align:right;"><a href="#top">top 🔝</a></p>

> **For Playlist**<br>
> pid, name, num_tracks, num_artists, num_albums, num_followers, num_edits, collaborative, duration_ms, modified_at, description<br>
> <br>
> **For Track**<br>
> *tid*, track_name, track_uri, artist_name, artist_uri, artist_name, album_uri, duration_ms<br>
> <br>
> **For Playlist & Track**<br>
> pid, *tid*, pos<br>

In [7]:
playlist_df = pd.DataFrame(
    columns=['pid', 'name', 
             'num_tracks', 'num_artists', 'num_albums', 
             'num_followers', 'num_edits', 
             'collaborative', 'duration_ms', 'modified_at', 'description']
)
track_df = pd.DataFrame(
    columns=['tid', 
             'track_name', 'track_uri', 
             'artist_name', 'artist_uri', 
             'album_name', 'album_uri', 
             'duration_ms']
)
playlist_track_df = pd.DataFrame(columns=['pid', 'tid', 'pos'])

In [8]:
pt_idx = 0

uri2id = dict()
uri2name = dict()

for p_idx, playlist in tqdm(enumerate(data['playlists'])):
    description = '' if 'description' not in playlist.keys() else playlist['description']
    playlist_df.loc[p_idx] = [
        playlist['pid'], playlist['name'], 
        playlist['num_tracks'], playlist['num_artists'], playlist['num_albums'],
        playlist['num_followers'], playlist['num_edits'],
        playlist['collaborative'], playlist['duration_ms'], playlist['modified_at'], description
    ]
    
    for track in playlist['tracks']:
        if track['track_uri'] not in uri2id.keys():
            uri2id[track['track_uri']] = len(uri2id)
            uri2name[track['track_uri']] = track['track_name']
            
            tid = uri2id[track['track_uri']]
            
            track_df.loc[tid] = [tid, 
                                 track['track_name'], track['track_uri'], 
                                 track['artist_name'], track['artist_uri'], 
                                 track['album_name'], track['album_uri'], 
                                 track['duration_ms']]
        
        tid = uri2id[track['track_uri']]
        assert track_df.loc[tid, 'track_name'] == track['track_name']   
        assert track_df.loc[tid, 'artist_name'] == track['artist_name']   
        assert track_df.loc[tid, 'album_name'] == track['album_name']   
        assert track_df.loc[tid, 'duration_ms'] == track['duration_ms']   

        playlist_track_df.loc[pt_idx] = [p_idx, tid, track['pos']]
        pt_idx += 1

1000it [11:56,  1.40it/s]


In [9]:
playlist_df

,pid,name,num_tracks,num_artists,num_albums,num_followers,num_edits,collaborative,duration_ms,modified_at,description
0,0,Throwbacks,52,37,47,1,6,false,11532414,1493424000,
1,1,Awesome Playlist,39,21,23,1,5,false,11656470,1506556800,
2,2,korean,64,31,51,1,18,false,14039958,1505692800,
3,3,mat,126,86,107,1,4,false,28926058,1501027200,
4,4,90s,17,16,16,2,7,false,4335282,1401667200,
...,...,...,...,...,...,...,...,...,...,...,...
995,995,old,41,36,40,1,8,false,9917901,1507852800,
996,996,Daze,17,15,17,1,13,false,3699248,1479254400,
997,997,rap,119,82,98,1,63,false,27538723,1410307200,
998,998,Country,108,40,75,1,37,false,24950143,1507939200,


In [10]:
track_df

,tid,track_name,track_uri,artist_name,artist_uri,album_name,album_uri,duration_ms
0,0,Lose Control (feat. Ciara & Fat Man Scoop),spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,The Cookbook,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863
1,1,Toxic,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,In The Zone,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800
2,2,Crazy In Love,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Dangerously In Love (Alben für die Ewigkeit),spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933
3,3,Rock Your Body,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,Justified,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266
4,4,It Wasn't Me,spotify:track:1lzr43nnXAijIGYnCT8M8H,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,Hot Shot,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600
...,...,...,...,...,...,...,...,...
34438,34438,I Don't Know,spotify:track:3uCHI1gfOUL5j5swEh0TcH,Jon D,spotify:artist:5HCypjplgh5uQezvBpOfXN,Roots,spotify:album:2KEQtuVl1cYsTYtVRUrNVi,189183
34439,34439,The Answer,spotify:track:0P1oO2gREMYUCoOkzYAyFu,Big Words,spotify:artist:0sHN89qak07mnug3LVVjzP,"Hollywood, a Beautiful Coincidence",spotify:album:5jrsRHRAmetu5e7RRBoxj7,263679
34440,34440,25.22,spotify:track:2oM4BuruDnEvk59IvIXCwn,Allan Rayman,spotify:artist:6Yv6OBXD6ZQakEljaGaDAk,Roadhouse 01,spotify:album:3CbNgBzI7r9o0F6VjH9sTY,189213
34441,34441,Good Feeling,spotify:track:4Ri5TTUgjM96tbQZd5Ua7V,Jon Jason,spotify:artist:77bNdkKYBBmc30CisCA6tE,Good Feeling,spotify:album:2dZ7oVNQBeLlpoUYfbEsJP,194720


In [11]:
playlist_track_df

,pid,tid,pos
0,0,0,0
1,0,1,1
2,0,2,2
3,0,3,3
4,0,4,4
...,...,...,...
67498,999,2076,39
67499,999,34439,40
67500,999,34440,41
67501,999,34441,42


In [12]:
playlist_df.to_csv('./data/mpd.slice.0-999.playlist.csv', index=False)
track_df.to_csv('./data/mpd.slice.0-999.track.csv', index=False)
playlist_track_df.to_csv('./data/mpd.slice.0-999.playlist.track.csv', index=False)